In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add
import cv2
import collections

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from Building_Vocab import load_data

### Load Data

In [5]:
li = load_data("./drive/My Drive/Files/variable_caption_processing.txt")
max_len = li[0]
vocab_size = li[1]

In [6]:
embedding_output = np.loadtxt("./drive/My Drive/Files/embedding_output.txt")

In [7]:
train_descriptions = load_data("./drive/My Drive/Files/train_descriptions.txt")

In [8]:
# Load the train images features from disk
with open("./drive/My Drive/Files/encoded_train_image_features.pkl", "rb") as encoded_pickle:
    encoding_train = pickle.load(encoded_pickle)

In [9]:
word_to_idx = load_data("./drive/My Drive/Files/word_to_idx.txt")

#### Model Architecture

In [10]:
# image feature extractor model
input_img_features = Input(shape=(2048,))
inp_img1 = Dropout(0.3)(input_img_features)
inp_img2 = Dense(256,activation='relu')(inp_img1)

In [11]:
# partial caption sequence model
input_captions = Input(shape=(max_len,))
inp_cap1 = Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_captions)
inp_cap2 = Dropout(0.3)(inp_cap1)
inp_cap3 = LSTM(256)(inp_cap2)

In [12]:
# decoder (feed forward) model
decoder1 = add([inp_img2,inp_cap3])
decoder2 = Dense(256,activation='relu')(decoder1)
outputs = Dense(vocab_size,activation='softmax')(decoder2)

# Combined Model
model = Model(inputs=[input_img_features,input_captions],outputs=outputs)

In [13]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 35, 50)       92400       input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
____________________________________________________________________________________________

In [14]:
model.layers[2].set_weights([embedding_output])
model.layers[2].trainable = False

In [15]:
model.compile(loss='categorical_crossentropy',optimizer="adam")

### Training of Model

#### Data Generator

In [16]:
def data_generator(train_descriptions, encoding_train, word_to_idx, max_len, num_photos_per_batch):

    X1, X2, y = [], [], []

    n=0

    while True:
        
        for key, desc_list in train_descriptions.items():
            n +=1

            photo = encoding_train[key+".jpg"]

            for desc in desc_list:
                
                seq = [ word_to_idx[word] for word in desc.split() if word in word_to_idx]  

                for i in range(1,len(seq)):

                    in_seq = seq[0:i]
                    out_seq = seq[i]

                    in_seq = pad_sequences([in_seq], maxlen=max_len, value=0, padding='post')[0]

                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)

            if n==num_photos_per_batch:
                yield [[np.array(X1), np.array(X2)], np.array(y)]
                X1, X2, y = [], [], []
                n=0

In [17]:
epochs = 20
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [18]:
## training
for i in range(epochs):
    generator = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('/content/drive/My Drive/model/model_' + str(i) + '.h5')

Epoch 1/1
2000/2000 [==============================] - 396s 198ms/step - loss: 4.3060
Epoch 1/1
2000/2000 [==============================] - 392s 196ms/step - loss: 3.5764
Epoch 1/1
2000/2000 [==============================] - 397s 198ms/step - loss: 3.3191
Epoch 1/1
2000/2000 [==============================] - 387s 194ms/step - loss: 3.1606
Epoch 1/1
2000/2000 [==============================] - 386s 193ms/step - loss: 3.0491
Epoch 1/1
2000/2000 [==============================] - 392s 196ms/step - loss: 2.9609
Epoch 1/1
2000/2000 [==============================] - 393s 196ms/step - loss: 2.8965
Epoch 1/1
2000/2000 [==============================] - 394s 197ms/step - loss: 2.8398
Epoch 1/1
2000/2000 [==============================] - 393s 197ms/step - loss: 2.7942
Epoch 1/1
2000/2000 [==============================] - 391s 195ms/step - loss: 2.7552
Epoch 1/1
2000/2000 [==============================] - 390s 195ms/step - loss: 2.7221
Epoch 1/1
2000/2000 [==============================] -

In [19]:
epochs = 30
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [20]:
## training
for i in range(20,epochs):
    generator = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('/content/drive/My Drive/model/model_' + str(i) + '.h5')

Epoch 1/1
1000/1000 [==============================] - 210s 210ms/step - loss: 2.4954
Epoch 1/1
1000/1000 [==============================] - 210s 210ms/step - loss: 2.4691
Epoch 1/1
1000/1000 [==============================] - 214s 214ms/step - loss: 2.4580
Epoch 1/1
1000/1000 [==============================] - 214s 214ms/step - loss: 2.4430
Epoch 1/1
1000/1000 [==============================] - 213s 213ms/step - loss: 2.4283
Epoch 1/1
1000/1000 [==============================] - 210s 210ms/step - loss: 2.4190
Epoch 1/1
1000/1000 [==============================] - 210s 210ms/step - loss: 2.4092
Epoch 1/1
1000/1000 [==============================] - 210s 210ms/step - loss: 2.3990
Epoch 1/1
1000/1000 [==============================] - 212s 212ms/step - loss: 2.3928
Epoch 1/1
1000/1000 [==============================] - 210s 210ms/step - loss: 2.3806
